In [16]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

In [17]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
     #replaces with our models dimensions if not openai
    name="rag", dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

In [18]:
import json 
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. Emily Carter',
  'subject': 'Introduction to Computer Science',
  'stars': 5,
  'review': 'Dr. Carter is an amazing professor! Her lectures are clear, and she always makes time for questions.'},
 {'professor': 'Dr. John Stevens',
  'subject': 'Calculus I',
  'stars': 4,
  'review': 'Great explanations, but the pace is a bit fast. Overall, a very knowledgeable professor.'},
 {'professor': 'Prof. Susan Reynolds',
  'subject': 'Modern Art History',
  'stars': 3,
  'review': 'Interesting lectures, but the grading is quite tough.'},
 {'professor': 'Dr. Alan Thompson',
  'subject': 'Organic Chemistry',
  'stars': 2,
  'review': "Content is challenging, but Dr. Thompson's teaching style is hard to follow."},
 {'professor': 'Dr. Jane Matthews',
  'subject': 'Psychology 101',
  'stars': 5,
  'review': 'Dr. Matthews is engaging and makes the material easy to understand. Highly recommend!'},
 {'professor': 'Prof. Michael Lee',
  'subject': 'World History',
  'stars': 4,
  'rev

In [20]:
processed_data = []
client = OpenAI()

#capture semantic info in numerical form

for review in data['reviews']:
    response = client.embeddings.create(
        input=review['review'],
        model="text-embedding-3-small",
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"]
        }
    })

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [22]:
processed_data[0]

IndexError: list index out of range

In [23]:
index = pc.Index('rag')
index.upsert(
    vectors = processed_data,
    namespace="ns1"
)

PineconeApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Fri, 23 Aug 2024 20:04:45 GMT', 'Content-Type': 'application/json', 'Content-Length': '74', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '49', 'x-pinecone-request-id': '3927314861368507717', 'x-envoy-upstream-service-time': '49', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"No vectors provided for upsert request","details":[]}


In [ ]:
index.describe_index_stats()